<p style="text-align:center; font-size:70px; font-weight:bold; color: greenyellow; border: 10px solid red; border-radius: 20px; background-color: black; padding: 30px 0px;">Random Forest Model</p>

## Step 1 : Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, learning_curve, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, log_loss

## Step 2 : Loading the dataset

In [ ]:
df = pd.read_csv("csi_data_amp_phase.csv")
print(df["Activity"].value_counts())

## Step 3 : Assigning data Indenpendent and Dependent variables

In [ ]:
X = df.drop(columns=['Activity','Timestamp']).values
y = df["Activity"]

## Step 4 : Applying Standard Scaler

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## Step 5 : Applying Label Encoder 

In [ ]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

## Step 6 : Class Mapping 

In [ ]:
print("Class Mapping:")
for i, class_label in enumerate(le.classes_):
    print(f"{i} -> {class_label}")

## Step 7 : Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Step 8 : Training the Random Forest Model

In [ ]:
rf_model = RandomForestClassifier(
    n_estimators=200,
    criterion='gini',
    max_depth=None,
    min_samples_split=11,
    class_weight='balanced_subsample',
    min_samples_leaf=11,
    random_state=42,
    max_features='log2',
)

rf_model.fit(X_train, y_train)

## Step 9 : Train Accuracy

In [ ]:
y_train_pred = rf_model.predict(X_train)
train_acc = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_acc:.4f}")

## Step 10 : Training Loss

In [ ]:
train_loss = log_loss(y_train, rf_model.predict_proba(X_train))
print(f"Training Loss: {train_loss:.4f}")

## Step 11 : Testing Accuracy

In [ ]:
y_test_pred = rf_model.predict(X_test)
test_acc = accuracy_score(y_test, y_test_pred)
print(f"Testing Accuracy: {test_acc:.4f}")

## Step 12 : Testing Loss

In [ ]:
test_loss = log_loss(y_test, rf_model.predict_proba(X_test))
print(f"Testing Loss: {test_loss:.4f}")

## Step 13 : Confusion Matrix

In [ ]:
y_pred = rf_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=sorted(y.unique()))
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Step 14 : Learning Curves

### Step 14 (a) : Accuracy

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(
    rf_model, X, y,
    cv=25,
    scoring='accuracy',
    n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 20),
    shuffle=True,
    random_state=42
)

train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

plt.figure(figsize=(6, 4))
plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training accuracy")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Validation accuracy")
plt.title("Learning Curve (Random Forest)")
plt.xlabel("Training examples")
plt.ylabel("Accuracy")
plt.legend(loc="best")
plt.grid(True)
plt.show()

### Step 14 (b) : Loss

In [ ]:
train_sizes, train_scores_loss, test_scores_loss = learning_curve(
    rf_model, X, y,
    cv=25,
    scoring='neg_log_loss',
    n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 20),
    shuffle=True,
    random_state=42
)

train_mean_loss = -np.mean(train_scores_loss, axis=1)
test_mean_loss = -np.mean(test_scores_loss, axis=1)

plt.figure(figsize=(6, 4))
plt.plot(train_sizes, train_mean_loss, 'o-', color='red', label='Training Loss')
plt.plot(train_sizes, test_mean_loss, 'o-', color='orange', label='Validation Loss')
plt.title('Loss Curve (Log Loss)')
plt.xlabel('Training Set Size')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

## Step 15 : Saving the model

In [ ]:
import joblib 
joblib.dump(scaler, "scaler.pkl")
joblib.dump(rf_model, "model.pkl")